<a href="https://colab.research.google.com/github/jqshang/CSC2541/blob/main/Dataload.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip -q install MDAnalysis MDAnalysisData pandas numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.9/108.9 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 50.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 54.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 2.7 MB/s eta 0:00:00


In [8]:
import MDAnalysis as mda
from MDAnalysisData import datasets
adk = datasets.fetch_adk_equilibrium()

In [9]:
print(adk.DESCR)

.. -*- coding: utf-8 -*-

.. _`adk-equilibrium-dataset`:

AdK equilibrium trajectory dataset

MD trajectory of apo adenylate kinase with CHARMM27 force field and
simulated with explicit water and ions in NPT at 300 K and 1
bar. Saved every 240 ps for a total of 1.004 µs. Produced on PSC
Anton. The trajectory only contains the protein while all solvent was
stripped in order to reduce the trajectory size. The whole protein was
superimposed on the CORE domain of AdK by RMSD fitting.

The topology is contained in the PSF file (CHARMM format). The
trajectory is contained in the DCD file (CHARMM/NAMD format).


Notes
-----

Data set characteristics:

 :size: 161 MB
 :number of trajectories: 1
 :number of frames:  4187
 :number of particles: 3341
 :creator: Sean Seyler
 :URL:  `10.6084/m9.figshare.5108170.v1 <https://doi.org/10.6084/m9.figshare.5108170.v1>`_
 :license: `CC-BY 4.0 <https://creativecommons.org/licenses/by/4.0/legalcode>`_
 :reference: [Seyler2017]_
    

.. [Seyler2017]  Seyler

In [14]:
adk.topology, adk.trajectory

('/root/MDAnalysis_data/adk_equilibrium/adk4AKE.psf',
 '/root/MDAnalysis_data/adk_equilibrium/1ake_007-nowater-core-dt240ps.dcd')

In [12]:
u = mda.Universe(adk.topology, adk.trajectory)

/usr/local/lib/python3.12/dist-packages/MDAnalysis/coordinates/DCD.py:171: DeprecationWarning: DCDReader currently makes independent timesteps by copying self.ts while other readers update self.ts inplace. This behavior will be changed in 3.0 to be the same as other readers. Read more at https://github.com/MDAnalysis/mdanalysis/issues/3889 to learn if this change in behavior might affect you.
  warnings.warn("DCDReader currently makes independent timesteps"


In [15]:
# Select all residues that have a Cα atom (standard proteins)
ca = u.select_atoms("protein and name CA")
n_res = ca.n_atoms
n_frames = len(u.trajectory)

In [18]:
n_res, n_frames

(214, 4187)

In [19]:
u.trajectory[0]

< Timestep 0 with unit cell dimensions [85.53471 85.53471 85.53471 90.      90.      90.     ] >

In [21]:
import numpy as np
import pandas as pd

In [22]:
ref = ca.positions.copy()

disp = np.empty((n_frames, n_res), dtype=np.float32)

for i in range(len(u.trajectory)):
    delta = ca.positions - ref
    disp[i, :] = np.linalg.norm(delta, axis=1)

disp

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [23]:
res_ids = [f"{r.resname}{r.resid}" for r in ca.residues]
df = pd.DataFrame(disp, columns=res_ids)
df.index.name = "frame"

In [24]:
df

,MET1,ARG2,ILE3,ILE4,LEU5,LEU6,GLY7,ALA8,PRO9,GLY10,...,VAL205,ARG206,ALA207,ASP208,LEU209,GLU210,LYS211,ILE212,LEU213,GLY214
frame,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4182,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4183,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4184,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
